# File Handling

In [6]:
import re

In [7]:
split_by = 1000
insert_str = 'INSERT INTO #employees VALUES \n'    
file_name = 'load_employees.sql'
out_file_name = 'load_employees_insert1000.sql'


with open(file_name, 'r') as file:
    rec_cnt = 0
    out_file = open(out_file_name, 'w')
    for line in file:
        rec_cnt += 1
        if rec_cnt == 1:
            out_file.write(insert_str)
            out_file.write(line)        
        elif rec_cnt == split_by:
            line = re.sub(r'\),', ')', line) # search for ), and replace with )
            out_file.write(line)
            out_file.write(';\n')
            rec_cnt = 0
        else:
            out_file.write(line)

out_file.close()

# Zoned Decimal Handling

In [3]:
import os 
import pandas as pd

In [4]:
def zoned_decimal_to_float(zoned_decimal):
	# Mapping of zoned decimal sign characters to their corresponding digits
	sign_map = {
		'{': '0', 'A': '1', 'B': '2', 'C': '3', 'D': '4',
		'E': '5', 'F': '6', 'G': '7', 'H': '8', 'I': '9',
		'}': '0', 'J': '1', 'K': '2', 'L': '3', 'M': '4',
		'N': '5', 'O': '6', 'P': '7', 'Q': '8', 'R': '9'
	}
	try:
		# Extract the last character to determine the sign and last digit
		last_char = zoned_decimal[-1]
		
		# Determine if the number is negative
		is_negative = last_char in '}JKLMNOPQR'
		
		# Get the last digit from the sign map
		last_digit = sign_map[last_char]
	
		# Combine the integer part with the last digit
		numeric_part = zoned_decimal[:-1] + last_digit
		
		# Convert to float
		result = float(numeric_part)
	
		# Apply the sign
		if is_negative:
			result = -result
	except KeyError as e:
		result = 0
		print(f"{e} Invalid zoned decimal format for {zoned_decimal}")		
	return result/100


In [ ]:
# Example usage
zoned_decimal = "0000001150{"
numeric_value = zoned_decimal_to_float(zoned_decimal)
print(numeric_value)  # Output: 1150.0

excel_dir = r"C:\Users\ny4007991\OneDrive - Munich Re\Professional\Munich Re"
excel_file = r"Imp Points Munich RE Mainframe.xlsx"
excel_path = os.path.join(excel_dir, excel_file)
pandas_df = pd.read_excel(excel_path,sheet_name="Temp1",header=None)
grossprem_F = pandas_df.iloc[:,5]
netprem_G = pandas_df.iloc[:,5]	
grossprem_F_float = grossprem_F.map(zoned_decimal_to_float)
netprem_G_float = netprem_G.map(zoned_decimal_to_float)
# print(pandas_df.head())
# print(grossprem_F.head())
# print(grossprem_F_float.head())
print(grossprem_F_float.values)
print("Gross Prem sum=",grossprem_F_float.sum())
print(netprem_G_float.head())
print("Net Prem sum=",netprem_G_float.sum())

# Excel

### ILegal Characters in Excel

In [8]:
import pandas as pd
import re
import os
import sys

In [ ]:
# Define a list of illegal characters for Excel
illegal_chars = ['','→']
# Create a regex pattern to match any of the illegal characters
illegal_chars_pattern = re.compile(f"[{''.join(re.escape(char) for char in illegal_chars)}]")
# print(illegal_chars_pattern)




In [11]:
"""
Explanation
contains_illegal_chars function:

This function checks if a given string contains any illegal characters defined by the regex pattern.
It returns True if illegal characters are found, otherwise False.
Reading the CSV file:

The CSV file is read into a DataFrame using pd.read_csv.
Applying the function:

The contains_illegal_chars function is applied to each cell in the DataFrame using applymap.
Filtering the DataFrame:

The DataFrame is filtered to get only the cells that contain illegal characters.
Additional Steps
Update the illegal_chars_pattern to include any other illegal characters you want to check for.
Run the script and verify the output to ensure it correctly identifies cells with illegal characters.
"""
def contains_illegal_chars(value):
    """
    Check if the value contains any illegal characters.
    """
    if isinstance(value, str):
        return bool(illegal_chars_pattern.search(value))
    return False

def remove_illegal_chars(value):
    """
    Remove illegal characters from the value.
    """
    if isinstance(value, str):
        # Define a regex pattern to match illegal characters
        # Replace illegal characters with an empty string
        return illegal_chars_pattern.sub('', value)
    return value



In [ ]:
# Demonstration of the functions
# Read the CSV file into a DataFrame
dir = r"C:\Users\ny4007991\OneDrive - Munich Re\Professional\Munich Re\Email Supp\Excel Dumps & Files\202407 July\RE Auto-Owners Cyber Suite 4355 and HCP 4632 inforce and bordereau files - June 2024"
file_nm = 'CBS4355CBORD.txt'
csv_path = os.path.join(dir, file_nm)
excel_df = pd.read_csv(csv_path, dtype=str,sep='~')

# # Apply the function to each cell in the DataFrame
# illegal_chars_df = df.applymap(extract_illegal_chars)
# # Filter out empty cells
# cells_with_illegal_chars = illegal_chars_df[illegal_chars_df.applymap(lambda x: x != '')]

# # Print the cells with illegal characters
# print(cells_with_illegal_chars)

# # Write the illegal characters to a text file
# output_file_path = os.path.join(dir, 'illegal_characters.txt')
# with open(output_file_path, 'w') as file:
#     for index, row in cells_with_illegal_chars.iterrows():
#         for cell in row:
#             file.write(str(cell) + '\n')

# print(f"Illegal characters have been written to {output_file_path}")

excel_file_nm = file_nm.replace('.txt', '.xlsx')
excel_path = dir + "\\" + excel_file_nm 

cleaned_df = excel_df.map(remove_illegal_chars)
# Find cells with illegal characters
illegal_cells = cleaned_df.map(contains_illegal_chars)
if illegal_cells.any().any():
    print("There are still illegal characters in the DataFrame.")
    print(cleaned_df[illegal_cells])
cleaned_df.to_excel(excel_path, index=False)
print(f"Conversion to excel successful {file_nm} to {excel_file_nm}")


### Color Coding for File Processing Status Munich Re in Excel

In [13]:
import openpyxl
from openpyxl.styles import PatternFill


In [14]:
def change_color(file_path):
    # Load the workbook and select the active sheet
    wb = openpyxl.load_workbook(file_path)
    ws = wb.active

    # Define the color mappings
    color_mappings = {
        "Files in test region and need automations.": "FFFF00",  # Yellow
        "Files not processed > 4 months": "FF0000",  # Red
        "Files not processed for more than 2 months and less than 4 months": "FFFF00",  # Yellow
        "Files which are in Run Off": "FF6600",  # Orange
        "Processed and all set in the system": "00FF00",  # Green
        "default": "CCCCCC"  # Grey for all other cases
    }

    # Get the last row in column M
    l_row = ws.max_row

    # Iterate through the cells in column M
    for row in range(2, l_row + 1):
        cell = ws[f'M{row}']
        value = cell.value
        color = color_mappings.get(value, color_mappings["default"])
        fill = PatternFill(start_color=color, end_color=color, fill_type="solid")
        
        # Apply the fill to the entire row
        for col in range(1, ws.max_column + 1):
            ws.cell(row=row, column=col).fill = fill

    # Save the workbook
    wb.save(file_path)



In [15]:
# Demonstration of the function
# Example usage
# \\cvgs070131\esplog\esp\UWdata\SIP\CIT_File_Report\File_Processing_report_CIT_Daily_Destination.xlsm
file_path = r'C:\Users\ny4007991\OneDrive - Munich Re\Professional\Munich Re\File_Processing_report_CIT_Daily_Destination.xlsm'
change_color(file_path)